In [146]:
from transformers import (pipeline, BertTokenizer, BertForMaskedLM, BertConfig, DataCollatorForLanguageModeling,
                            Trainer, TrainingArguments, RobertaTokenizer, RobertaForMaskedLM)
import torch
import numpy as np
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets
from tqdm import tqdm


In [19]:

unmasker = pipeline('fill-mask', model='bert-base-uncased')

unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731099545955658,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i'm a fashion model."},
 {'score': 0.08774477988481522,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.05338403955101967,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.04667219892144203,
  'token': 3565,
  'token_str': 'super',
  'sequence': "hello i'm a super model."},
 {'score': 0.02709587849676609,
  'token': 2986,
  'token_str': 'fine',
  'sequence': "hello i'm a fine model."}]

In [52]:
question = "Le Journal de Mickey is written in [MASK]."
unmasker(question)

[{'score': 0.9272459745407104,
  'token': 2413,
  'token_str': 'french',
  'sequence': 'le journal de mickey is written in french.'},
 {'score': 0.013484428636729717,
  'token': 2394,
  'token_str': 'english',
  'sequence': 'le journal de mickey is written in english.'},
 {'score': 0.012427841313183308,
  'token': 16659,
  'token_str': 'breton',
  'sequence': 'le journal de mickey is written in breton.'},
 {'score': 0.004832481034100056,
  'token': 13973,
  'token_str': 'catalan',
  'sequence': 'le journal de mickey is written in catalan.'},
 {'score': 0.0039960830472409725,
  'token': 3000,
  'token_str': 'paris',
  'sequence': 'le journal de mickey is written in paris.'}]

In [128]:
question = "[CLS] national city corp.'s headquarters are in [MASK]. [SEP]"
unmasker(question)

[{'score': 0.03792567923665047,
  'token': 3190,
  'token_str': 'chicago',
  'sequence': "national city corp.'s headquarters are in chicago."},
 {'score': 0.03339061141014099,
  'token': 2899,
  'token_str': 'washington',
  'sequence': "national city corp.'s headquarters are in washington."},
 {'score': 0.02700265869498253,
  'token': 9506,
  'token_str': 'indianapolis',
  'sequence': "national city corp.'s headquarters are in indianapolis."},
 {'score': 0.025303542613983154,
  'token': 11334,
  'token_str': 'minneapolis',
  'sequence': "national city corp.'s headquarters are in minneapolis."},
 {'score': 0.024576237425208092,
  'token': 5862,
  'token_str': 'seattle',
  'sequence': "national city corp.'s headquarters are in seattle."}]

In [19]:
config = BertConfig.from_pretrained('bert-base-uncased')
model_path = "best_model.pt"
state_dict = torch.load(model_path)
model = BertForMaskedLM(config)
model.load_state_dict(state_dict)
model.eval()

RuntimeError: Error(s) in loading state_dict for BertForMaskedLM:
	Missing key(s) in state_dict: "cls.predictions.bias", "cls.predictions.transform.dense.weight", "cls.predictions.transform.dense.bias", "cls.predictions.transform.LayerNorm.weight", "cls.predictions.transform.LayerNorm.bias", "cls.predictions.decoder.weight", "cls.predictions.decoder.bias". 
	Unexpected key(s) in state_dict: "classifier.weight", "classifier.bias", "bert.pooler.dense.weight", "bert.pooler.dense.bias". 

In [ ]:
text = "The quick brown [MASK] jumps over the lazy dog."
tokenized_text = text.split()
tokenized_text[3] = '[MASK]'
masked_text = ' '.join(tokenized_text)

input = tokenizer.encode(masked_text, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:

    print(tokenizer.decode(token))




c a t
d o g
b e a r
m a n
# # i e


In [20]:
from transformers import BertTokenizer, BertForMaskedLM
from sklearn.metrics import accuracy_score
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the model's predictions
predicted_words = []

for sentence in sentences:
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Use the model to predict the masked token
    outputs = model(**inputs)
    predictions = outputs.logits[0, mask_index, :]

    # Get the predicted token
    predicted_token = torch.argmax(predictions)
    predicted_word = tokenizer.decode([predicted_token])

    # Add the predicted word to the list
    predicted_words.append(predicted_word)

# Calculate the accuracy of the model's predictions
accuracy = accuracy_score(actual_words, predicted_words)

print(f"Accuracy: {accuracy}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 0.0


In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the losses
losses = []

for sentence, actual_word in zip(sentences, actual_words):
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Replace the masked token with the actual token
    inputs["input_ids"][0, mask_index] = tokenizer.encode(actual_word, add_special_tokens=False)[0]

    # Use the model to calculate the loss
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

    # Add the loss to the list
    losses.append(loss.item())

# Calculate the average loss
average_loss = sum(losses) / len(losses)

print(f"Average Loss: {average_loss}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Loss: 3.7346343994140625


In [5]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [6]:
wiki_dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [7]:
wiki_dataset["train"][4]["text"]

" The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game 's opening theme was sung by May 'n . \n"

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


In [2]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

class TrainerWithCustomLoss(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

        # if loss is nan, there is an issue with the labels, we return 0
        if torch.isnan(loss):
            loss = torch.tensor(0.0, requires_grad=True).to(loss.device)
        return (loss, outputs) if return_outputs else loss
    """
    def compute_loss(self, model, inputs, return_outputs=False):
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)

        # code for calculating accuracy
        if "labels" in inputs:
            logits = outputs.logits.detach()
            preds = logits.argmax(dim=1)
            acc1 = metric.compute(predictions=preds, references=inputs.labels)
            self.log({'accuracy_score': acc1})
            acc = (
                (preds.argmax(axis=-1) == inputs.labels.reshape(1, len(inputs.labels))[0])
                .type(torch.float)
                .mean()
                .item()
            )
            self.log({"train_accuracy": acc})
        # end code for calculating accuracy
                    
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss
    """


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator
)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

NameError: name 'model' is not defined

In [23]:
tokenizer = BertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = BertForMaskedLM.from_pretrained('distilbert/distilbert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', '

In [24]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

{'eval_loss': 10.404870986938477,
 'eval_runtime': 210.8167,
 'eval_samples_per_second': 20.672,
 'eval_steps_per_second': 1.295}

# Try to make BERT degenerate

In [26]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


# training with high lr
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-2,
    num_train_epochs=1
)


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    train_dataset=wiki_dataset_tokenized["train"].select(range(100)),
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator

)

# get loss on test set
trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss


KeyboardInterrupt: 

In [5]:

unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

preds = unmasker("Hello I'm a [MASK] model.")

# Without Trainer

In [2]:
import numpy as np

wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
test_text = wiki_dataset["test"]["text"]

np.random.seed(42)

# discard empty text and with less than 100 characters
test_text = [t for t in test_text if len(t) > 100]

# sample 100 texts
test_text = np.random.choice(test_text, 100)

# mask 1 word in each text
mask_indices = [np.random.randint(0, len(t.split())) for t in test_text]


# mask the tokens
masked_text = []
for i, t in enumerate(test_text):
    tokenized_text = t.split()
    tokenized_text[mask_indices[i]] = '[MASK]'
    masked_text.append(' '.join(tokenized_text))

for i in range(10):
    print("Text: ")
    print(masked_text[i])

Text: 
Writing in Variety , Todd McCarthy said the cast ensemble " could not be better " ; he praised Spacey 's " handling of innuendo , subtle sarcasm , and blunt talk " and the way he imbued Lester with " genuine feeling " . Janet Maslin in The New York Times said Spacey was at his " wittiest and most agile " to date , and Roger Ebert of the Chicago Sun @-@ Times singled Spacey out for successfully portraying a man who " does reckless and foolish things [ but who ] doesn 't deceive himself " . Kevin Jackson of Sight & Sound said Spacey impressed in ways distinct from his previous performances , the most satisfying aspect being his portrayal of " both sap and hero " . Writing in Film Quarterly , Gary Hentzi praised the actors , but said that characters such as Carolyn and Col. Fitts were stereotypes . Hentzi accused Mendes and Ball of identifying too readily with Jane and Ricky , saying the latter was their " fantasy figure " — a teenaged boy who 's an absurdly wealthy artist able to 

In [17]:
# compute loss with reference model on sampled texts
losses = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
reference_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
reference_model.eval()
with torch.no_grad():
    for i, ref_text in enumerate(test_text):

        # mask a word in the text randomly
        masked_text = ref_text.split()
        masked_text[mask_indices[i]] = '[MASK]'
        masked_text = ' '.join(masked_text)

        inputs = tokenizer(masked_text, return_tensors='pt')

        #masked_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        # see example in: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForMaskedLM

        print("inputs['input_ids'] shape: ", inputs["input_ids"].shape)

        labels = tokenizer(ref_text, return_tensors='pt')["input_ids"]
        print("labels shape: ", labels.shape)
        print("ref_text: ", ref_text)
        print("masked_text: ", masked_text)
        print("len(masked_text.split()): ", len(masked_text.split()))
        print("len(ref_text.split()): ", len(ref_text.split()))
        labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)


        #mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        outputs = reference_model(**inputs, labels=labels)
        logits = outputs.logits
        top_5_preds = torch.topk(logits[0, mask_index, :], 5, dim=1).indices[0].tolist()

        print("Masked text: ", masked_text)
        print("Top 5 predictions: ", tokenizer.decode(top_5_preds))
        print("Actual word: ", ref_text.split()[mask_indices[i]])
        print("")
        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


inputs['input_ids'] shape:  torch.Size([1, 354])
labels shape:  torch.Size([1, 354])
ref_text:   Writing in Variety , Todd McCarthy said the cast ensemble " could not be better " ; he praised Spacey 's " handling of innuendo , subtle sarcasm , and blunt talk " and the way he imbued Lester with " genuine feeling " . Janet Maslin in The New York Times said Spacey was at his " wittiest and most agile " to date , and Roger Ebert of the Chicago Sun @-@ Times singled Spacey out for successfully portraying a man who " does reckless and foolish things [ but who ] doesn 't deceive himself " . Kevin Jackson of Sight & Sound said Spacey impressed in ways distinct from his previous performances , the most satisfying aspect being his portrayal of " both sap and hero " . Writing in Film Quarterly , Gary Hentzi praised the actors , but said that characters such as Carolyn and Col. Fitts were stereotypes . Hentzi accused Mendes and Ball of identifying too readily with Jane and Ricky , saying the latte

RuntimeError: The size of tensor a (39) must match the size of tensor b (41) at non-singleton dimension 1

In [13]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.88

- run 1: loss = 0.65
- run 2: loss = 0.65

In [10]:
# compute loss with reference model on sampled texts
losses = []

# use distilbert as a placeholder for the finetuned_model
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
finetuned_model = BertForMaskedLM.from_pretrained('distilbert-base-uncased')
finetuned_model.eval()
with torch.no_grad():
    for i, t in enumerate(masked_text):
        inputs = tokenizer.encode_plus(t, return_tensors='pt')
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        outputs = finetuned_model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.

Average Loss: 10.491268739700317


- run 1: loss = 10.49
- run 2: loss = 10.49

In [28]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


# training with high lr
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-2,
    num_train_epochs=1
)


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    train_dataset=wiki_dataset_tokenized["train"].select(range(100)),
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator

)

# compute loss with reference model on sampled texts
losses = []

# use distilbert as a placeholder for the finetuned_model
finetuned_model = model
finetuned_model.eval()
with torch.no_grad():
    for i, t in enumerate(masked_text):
        inputs = tokenizer.encode_plus(t, return_tensors='pt').to(finetuned_model.device)
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        outputs = finetuned_model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Loss: 0.6526461672782898


# Test with fact completion dataset

## Bert and Distilbert

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
reference_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [26]:
fact_completion_dataset = load_dataset('Polyglot-or-Not/Fact-Completion')

Generating Bulgarian split:   0%|          | 0/20577 [00:00<?, ? examples/s]

Generating Catalan split:   0%|          | 0/18898 [00:00<?, ? examples/s]

Generating Croatian split:   0%|          | 0/7358 [00:00<?, ? examples/s]

Generating Czech split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating Danish split:   0%|          | 0/23365 [00:00<?, ? examples/s]

...

Generating Serbian split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating Slovenian split:   0%|          | 0/7873 [00:00<?, ? examples/s]

Generating Spanish split:   0%|          | 0/18786 [00:00<?, ? examples/s]

Generating Swedish split:   0%|          | 0/21576 [00:00<?, ? examples/s]

Generating Ukrainian split:   0%|          | 0/7918 [00:00<?, ? examples/s]

In [28]:
fact_completion_dataset["English"]

Dataset({
    features: ['dataset_id', 'stem', 'true', 'false', 'relation', 'subject', 'object'],
    num_rows: 26254
})

In [156]:
fact_completion_dataset = load_dataset('Polyglot-or-Not/Fact-Completion')["English"]

# filter out questions where the answer is not a single word for the tokenizer
len_before = len(fact_completion_dataset)
#fact_completion_dataset = fact_completion_dataset.filter(lambda x: len(x["true"].split()) == 1)
fact_completion_dataset = fact_completion_dataset.filter(lambda x: len(tokenizer(x["true"]).input_ids) == 3)
len_after = len(fact_completion_dataset)
print(f"Filtered out {len_before - len_after} questions out of {len_before}")

# take 100 random samples from the dataset
fact_completion_dataset = fact_completion_dataset.shuffle(seed=40)
fact_completion_dataset = fact_completion_dataset.select(range(1000))
questions = fact_completion_dataset["stem"]
answers = fact_completion_dataset["true"]

# add [MASK] at the end of each question
# we also add a ".", this is very important for bert models
questions_masked = [q + " [MASK]." for q in questions]

for i in range(10):
    print("Question: ")
    print(questions_masked[i])
    print("Answer: ")
    print(answers[i])

Filtered out 4265 questions out of 26254
Question: 
Laisse tomber les filles is written in [MASK].
Answer: 
French
Question: 
Albert Calmette died at [MASK].
Answer: 
Paris
Question: 
Eric Partridge writes in [MASK].
Answer: 
English
Question: 
The headquarter of Freud Communications is located in [MASK].
Answer: 
London
Question: 
The Suicide File, that was started in [MASK].
Answer: 
Boston
Question: 
Bernardo Salviati, who holds the position of [MASK].
Answer: 
cardinal
Question: 
Shirley Scott, which is represented by [MASK].
Answer: 
Prestige
Question: 
San Francisco Conservatory of Music, in [MASK].
Answer: 
California
Question: 
Kevin Garnett is a professional [MASK].
Answer: 
basketball
Question: 
Albert Sharpe's occupation is [MASK].
Answer: 
actor


In [157]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
reference_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

losses = []
reference_model.eval()

with torch.no_grad():
    for i, question in enumerate(questions_masked):

        # see example in: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForMaskedLM
        inputs = tokenizer(question, return_tensors='pt')

        question_with_answer = questions[i] + " " + answers[i] + "."

        print("question: ", question)
        labels = tokenizer(question_with_answer, return_tensors='pt')["input_ids"]
        labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

        outputs = reference_model(**inputs, labels=labels)
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)
        top_5_preds = torch.topk(outputs.logits[0, mask_index, :], 5, dim=1).indices[0].tolist()

        print("Top 5 predictions: ", tokenizer.decode(top_5_preds))
        print("Actual answer: ", answers[i])
        loss = outputs.loss
        losses.append(loss.item())
        print("loss: ", loss.item())
        print("")

print(f"Average Loss: {sum(losses) / len(losses)}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


question:  Laisse tomber les filles is written in [MASK].
Top 5 predictions:  french english italian breton german
Actual answer:  French
loss:  0.0356057770550251

question:  Albert Calmette died at [MASK].
Top 5 predictions:  home paris cannes montreal versailles
Actual answer:  Paris
loss:  2.688735008239746

question:  Eric Partridge writes in [MASK].
Top 5 predictions:  english london french fiction poetry
Actual answer:  English
loss:  1.2522093057632446

question:  The headquarter of Freud Communications is located in [MASK].
Top 5 predictions:  london paris berlin vienna dublin
Actual answer:  London
loss:  1.0680876970291138

question:  The Suicide File, that was started in [MASK].
Top 5 predictions:  london chicago april california england
Actual answer:  Boston
loss:  4.370449066162109

question:  Bernardo Salviati, who holds the position of [MASK].
Top 5 predictions:  fr hon bishop archbishop prof
Actual answer:  cardinal
loss:  4.531750202178955

question:  Shirley Scott, 

In [85]:
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
reference_model = BertForMaskedLM.from_pretrained('distilbert-base-uncased')

losses = []
reference_model.eval()

with torch.no_grad():
    for i, question in enumerate(questions_masked):

        # see example in: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForMaskedLM
        inputs = tokenizer(question, return_tensors='pt')

        question_with_answer = questions[i] + " " + answers[i] + "."

        print("question: ", question)
        labels = tokenizer(question_with_answer, return_tensors='pt')["input_ids"]
        labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

        outputs = reference_model(**inputs, labels=labels)
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)
        top_5_preds = torch.topk(outputs.logits[0, mask_index, :], 5, dim=1).indices[0].tolist()

        print("Top 5 predictions: ", tokenizer.decode(top_5_preds))
        print("Actual answer: ", answers[i])
        loss = outputs.loss
        losses.append(loss.item())
        print("loss: ", loss.item())
        print("")

print(f"Average Loss: {sum(losses) / len(losses)}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attentio

question:  National City Corp.'s headquarters are in [MASK].
Top 5 predictions:  relentlessea bernardino subcommittee tod
Actual answer:  Cleveland
loss:  10.695258140563965

question:  Suriname shares a border with [MASK].
Top 5 predictions:  ##eaape bernardino relentless miranda
Actual answer:  Brazil
loss:  9.399699211120605

question:  The headquarter of Amirkabir University of Technology is in [MASK].
Top 5 predictions:  shining stretched emmy showcasing miranda
Actual answer:  Tehran
loss:  10.498001098632812

question:  County of London is within [MASK].
Top 5 predictions:  ##ea bernardino worlds miranda natives
Actual answer:  England
loss:  9.999478340148926

question:  Haji Mastan passed away in [MASK].
Top 5 predictions:  jamestown emmy worlds norfolk entertained
Actual answer:  Bombay
loss:  9.851590156555176

question:  Tampines Expressway is located in [MASK].
Top 5 predictions:  rezaea shining denies worlds
Actual answer:  Singapore
loss:  10.59842300415039

question:  S

## Roberta

In [94]:
tokenizer(fact_completion_dataset[0]["true"])

{'input_ids': [0, 32841, 2], 'attention_mask': [1, 1, 1]}

In [108]:
tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-large')
reference_model = RobertaForMaskedLM.from_pretrained('FacebookAI/roberta-large')

fact_completion_dataset = load_dataset('Polyglot-or-Not/Fact-Completion')["English"]

# filter out questions where the answer is not a single word for the tokenizer
len_before = len(fact_completion_dataset)
#fact_completion_dataset = fact_completion_dataset.filter(lambda x: len(x["true"].split()) == 1)
fact_completion_dataset = fact_completion_dataset.filter(lambda x: len(tokenizer(x["true"]).input_ids) == 3)
len_after = len(fact_completion_dataset)
print(f"Filtered out {len_before - len_after} questions out of {len_before}")

# take 100 random samples from the dataset
fact_completion_dataset = fact_completion_dataset.shuffle(seed=41)
fact_completion_dataset = fact_completion_dataset.select(range(1000))
questions = fact_completion_dataset["stem"]
answers = fact_completion_dataset["true"]

# add [MASK] at the end of each question
# we also add a ".", this is very important for bert models
questions_masked = [q + " <mask>." for q in questions]

for i in range(10):
    print("Question: ")
    print(questions_masked[i])
    print("Answer: ")
    print(answers[i])

Filtered out 16415 questions out of 26254
Question: 
Marc Bloch writes in <mask>.
Answer: 
French
Question: 
Martin Drennan's position is <mask>.
Answer: 
bishop
Question: 
Daniel Santiago professionally plays the sport <mask>.
Answer: 
basketball
Question: 
Europa-Park was named for <mask>.
Answer: 
Europe
Question: 
Russian Post, whose headquarters are in <mask>.
Answer: 
Moscow
Question: 
The law in Cossonay declares the language <mask>.
Answer: 
French
Question: 
Valeria Bruni Tedeschi spoke the language <mask>.
Answer: 
French
Question: 
Grace and Frankie originally aired on <mask>.
Answer: 
Netflix
Question: 
Ducks Deluxe, founded in <mask>.
Answer: 
London
Question: 
Malcolm was native to <mask>.
Answer: 
Scotland


In [109]:
losses = []
reference_model.eval()

with torch.no_grad():
    for i, question in enumerate(questions_masked):

        # see example in: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForMaskedLM
        inputs = tokenizer(question, return_tensors='pt')

        question_with_answer = questions[i] + " " + answers[i] + "."

        print("question: ", question)
        
        labels = tokenizer(question_with_answer, return_tensors='pt')["input_ids"]
        labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

        outputs = reference_model(**inputs, labels=labels)
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)
        top_5_preds = torch.topk(outputs.logits[0, mask_index, :], 5, dim=1).indices[0].tolist()

        print("Top 5 predictions: ", tokenizer.decode(top_5_preds))
        print("Actual answer: ", answers[i])
        loss = outputs.loss
        losses.append(loss.item())
        print("loss: ", loss.item())
        print("")

print(f"Average Loss: {sum(losses) / len(losses)}")



question:  Marc Bloch writes in <mask>.
Top 5 predictions:   Slate Salon Commentary Politico Forbes
Actual answer:  French
loss:  8.411077499389648

question:  Martin Drennan's position is <mask>.


Top 5 predictions:   unchanged clear here this unclear
Actual answer:  bishop
loss:  15.310789108276367

question:  Daniel Santiago professionally plays the sport <mask>.
Top 5 predictions:   football soccer basketball volleyball boxing
Actual answer:  basketball
loss:  2.0549755096435547

question:  Europa-Park was named for <mask>.
Top 5 predictions:   him her Merkel Macron it
Actual answer:  Europe
loss:  5.361565589904785

question:  Russian Post, whose headquarters are in <mask>.
Top 5 predictions:   Moscow London Paris Russia Beijing
Actual answer:  Moscow
loss:  0.08291833847761154

question:  The law in Cossonay declares the language <mask>.
Top 5 predictions:   taboo illegal official forbidden endangered
Actual answer:  French
loss:  7.78788948059082

question:  Valeria Bruni Tedeschi spoke the language <mask>.
Top 5 predictions:   well beautifully herself perfectly differently
Actual answer:  French
loss:  4.728906631469727

question:  Grace and Frankie originally aired on <m

In [151]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
reference_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

batch_size = 4
batches_questions_masked = [questions_masked[i:i + batch_size] for i in range(0, len(questions_masked), batch_size)]

question_with_answers = [questions[i] + " " + answers[i] + "." for i in range(len(questions))]
batches_answers = [question_with_answers[i:i + batch_size] for i in range(0, len(question_with_answers), batch_size)]


batches = list(zip(batches_questions_masked, batches_answers))

losses = []
reference_model.eval()

with torch.no_grad():
    for batch_question, batch_answer in tqdm(batches, desc="Answering fact completion questions..."):
    #for batch_question, batch_answer in batches:
        inputs = tokenizer(batch_question, return_tensors='pt', padding=True, truncation=True)

        questions_with_answer = batch_answer
        #questions_with_answer = [questions[i] + " " + answers[i] + "." for i in range(len(batch))]

        labels = tokenizer(questions_with_answer, return_tensors='pt', padding=True, truncation=True)["input_ids"]
        labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

        outputs = reference_model(**inputs, labels=labels)
        mask_indices = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)

        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Answering fact completion questions...: 100%|██████████| 250/250 [00:28<00:00,  8.85it/s]

Average Loss: 4.874574433088303


# Testing JudgeLM